<a href="https://colab.research.google.com/github/Brennan-Richards/Research-Recommendation/blob/main/Academic_Research_Recommendation_FT%5E2_GPT2_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/')
# os.chdir('content/drive/MyDrive/CSC561')
os.chdir('content/drive/Shareddrives/CSC561 Finals')

In [ ]:
!ls

 brennan-richards			  gpt2-finetuned-academic-topics
'Choosing Final Project Framework.gdoc'   permuted_tags_by_user.csv
'CiteULike Dataset.gdoc'		  tags_by_user.csv
 clean_tags.csv				  tags.dat
'Final Project Dataset Engineering'	  tags_df_w_embed_dist.csv
'Fine Tuning GPT-2 HuggingFace'		  thumbnail-temp.png
 flagged				  users.dat
'FT^2 GPT2 Model.ipynb'


In [ ]:
!pip3 install trl
!pip install -U transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.3.2-py3-none-any.whl (1.8 MB)
  Using cached tokenizers-0.10.3-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.2
    Uninstalling tokenizers-0.13.2:
      Successfully uninstalled tokenizers-0.13.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.25.1
    Uninstalling transformers-4.25.1:
      Successfully uninstalled transformers-4.25.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.25.1-py3-none-any.whl (5.8 MB)
  Using cached tokenizers-0.13.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Unin

In [ ]:
!pip install huggingface_hub
!apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [ ]:
!pip install -U gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached gradio-3.16.2-py3-none-any.whl (14.2 MB)
  Attempting uninstall: gradio
    Found existing installation: gradio 2.5.0
    Uninstalling gradio-2.5.0:
      Successfully uninstalled gradio-2.5.0


In [ ]:
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import tensorflow as tf
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel
from trl.gpt2 import GPT2HeadWithValueModel, respond_to_batch
from trl.ppo import PPOTrainer

### Test the model

In [ ]:
model_path = 'brennan-richards/gpt2-finetuned-academic-topics'


In [ ]:
from transformers import pipeline, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('gpt2')
test_model = GPT2LMHeadModel.from_pretrained(model_path, from_tf=True)
generator1 = pipeline(task="text-generation", model=test_model, tokenizer=tokenizer)



All TF 2.0 model weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.


In [ ]:
i = "deep learning, machine learning, genomics, neuroscience,"
max_len = len(i.split(" ")) + 10
print(max_len)
output_list = generator1(i, num_return_sequences=1, max_length=max_len ) # Set max length as the len of the input + (len of largest token in dataset)
output_list

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


16


[{'generated_text': 'deep learning, machine learning, genomics, neuroscience, information soc, network classification'}]

In [ ]:
# NOTE: Test output of non-RL versus RL model.

# from transformers import logging

# logging.set_verbosity_error()

# max_len = len(i.split(" ")) + 10
# print(max_len)

# tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
# test_model = GPT2LMHeadModel.from_pretrained(model_path, from_tf=True)
# generator1 = pipeline(task="text-generation", model=test_model, tokenizer=tokenizer)
# output1 = generator1(i, num_return_sequences=1, max_length=max_len ) # Set max length as the len of the input + (len of largest token in dataset)
# print(output1, '\n')

# # tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
# gpt2_tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
# gpt2_model = GPT2HeadWithValueModel.from_pretrained(model_name)
# query_tensor = gpt2_tokenizer.encode(i, return_tensors="pt")
# response_tensor  = respond_to_batch(gpt2_model, query_tensor, txt_len= max_len )
# response_txt = gpt2_tokenizer.decode(response_tensor[0,:])
# print(gpt2_model.v_head.detach_head)
# print(response_txt, '\n')

# NOTE: W/ detached value head. This is mostly for training.
# gpt2_model.detach_value_head()
# response_tensor  = respond_to_batch(gpt2_model, query_tensor, txt_len= max_len )
# response_txt = gpt2_tokenizer.decode(response_tensor[0,:])
# print(gpt2_model.v_head.detach_head)
# print(response_txt, '\n')

### Reinforcement learning with TRL package

In [ ]:
import requests
from PIL import Image
from io import BytesIO
import IPython
import urllib
import gradio as gr
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
# import plotly.express as px
import random

In [ ]:
print(gr.__version__)

3.16.2


In [ ]:
model_name = model_path

i = "deep learning, machine learning, genomics,"

# get models
gpt2_model = GPT2HeadWithValueModel.from_pretrained(model_name)
gpt2_model_ref = GPT2HeadWithValueModel.from_pretrained(model_name)
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at brennan-richards/gpt2-finetuned-academic-topics and are newly initialized: ['v_head.summary.bias', 'v_head.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at brennan-richards/gpt2-finetuned-academic-topics and are newly initialized: ['v_head.summary.bias', 'v_head.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Move models to GPU for training
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# _ = gpt2_model.to(device)
# _ = gpt2_model_ref.to(device)
# print(f"Using device: {device}")

In [ ]:
# img_response = requests.get("https://upload.wikimedia.org/wikipedia/commons/thumb/8/89/Gray754.png/77px-Gray754.png", stream=True)
# img = Image.open(img_response.raw)

In [ ]:
# get models
gpt2_model = GPT2HeadWithValueModel.from_pretrained(model_name)
gpt2_model_ref = GPT2HeadWithValueModel.from_pretrained(model_name)
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# gpt2_model.detach_value_head()
print("DETACH HEAD = " , gpt2_model.v_head.detach_head)


Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at brennan-richards/gpt2-finetuned-academic-topics and are newly initialized: ['v_head.summary.bias', 'v_head.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at brennan-richards/gpt2-finetuned-academic-topics and are newly initialized: ['v_head.summary.bias', 'v_head.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DETACH HEAD =  False


In [ ]:

# initialize trainer
ppo_config = {'batch_size': 1, 'forward_batch_size': 1, "lr": 1.41e-5, "ppo_epochs": 1} # What is the PPO_epochs param?
ppo_trainer = PPOTrainer(gpt2_model, gpt2_model_ref, gpt2_tokenizer, **ppo_config)

# Gradio wants all models on CPU.
device = torch.device("cpu")
_ = gpt2_model.to(device)
_ = gpt2_model_ref.to(device)
assert str(device) == 'cpu'

# Stats
n_step = 0
rewards = []
stats_df = pd.DataFrame({'n_steps':[], 'rewards':[]})

# Default interests
perm = np.random.permutation( "deep learning, computer vision, genomics, statistics, chemistry, evolution".split(", ") )
default_interests = ', '.join(perm[:4]) + ','
print("INTERESTS: ", default_interests)

# Information from the previous step
last_query_tensor, last_response_tensor = torch.empty((1, 1)), torch.empty((1, 1))

def interact(query_txt, last_step_reward=None, n_concepts=3, n_works=3):
  """
  Wrapper for querying model, passing data to OpenAlex, and retrieving reward from user.

  :last_step_reward = the reward from step N - 1

  IMPORTANT: Are the model parameters being saved?

  NOTE: Model seems to be diverging, try decreasing learning rate
  """

  # Train model with ppo. Runs an Adam optimization step on the model weights.
  train_stats = []
  fig = plt.figure()
  global n_step
  global stats_df
  global rewards
  global last_query_tensor
  global last_response_tensor
  if not n_step == 0 and n_step % 2 == 0: # Don't run this on the first step,
    print("REWARD = ", last_step_reward)

    # Track rewards over time for plotting.
    rewards.append(last_step_reward)

    # TODO (A): Create a NN that models the implied reward function based on the user's input.

    # TODO: Use (A) to fine tune GPT2 model with RL offline for a number of steps and return to this function.

    print("Updating model params")
    train_stats = ppo_trainer.step(last_query_tensor, last_response_tensor, [last_step_reward])
    #print("TRAIN STATS:", train_stats)

    stats_df = pd.DataFrame.from_dict( { "n_steps": range(1, (len(rewards) + 1) ), "rewards": rewards } )
    print(stats_df)
    plt.plot(stats_df['n_steps'], stats_df['rewards'])
    plt.title("Rewards over time")
    plt.ylabel("rewards")
    plt.xlabel("# of human interactions")
  n_step += 1

  # encode a query
  query_tensor = gpt2_tokenizer.encode(query_txt, return_tensors="pt")

  # Use GPT-2's next generated topic to retrieve n_concepts
  top_n_concepts = []
  while len(top_n_concepts) == 0: # Loop until getting a valid response from OpenAlex
    # get model response
    response_tensor  = respond_to_batch(gpt2_model, query_tensor, txt_len= ( len(query_txt.split(" ")) + 10) )
    response_txt = gpt2_tokenizer.decode(response_tensor[0,:])
    print(query_txt + response_txt)

    # Parse the suggestion out of the generator pipeline
    recommendation = (query_txt + response_txt).split(',')[len(query_txt.split(","))]
    print("RECOMMENDATION = ", recommendation)

    # Return a list of concepts related to the search term
    url = f"https://api.openalex.org/concepts?search={ recommendation }&sort=relevance_score:desc"
    # print(url)
    r_json = requests.get(url).json()
    while r_json is None: # IMPORTANT: Avoid error in API call by not returning None
      r_json = requests.get(url).json()

    try:
      num_concepts = min(n_concepts + 1, len(r_json["results"]))
      top_n_concepts = r_json["results"][0:n_concepts + 1]
      continue
    except Exception: # train model with ppo, output was not found in OpenAlex
      auto_reward = -1
      print(f"Auto-param update. Reward = {auto_reward} ")
      train_stats = ppo_trainer.step(query_tensor, response_tensor, auto_reward)
      continue

  # Format the top N concepts
  formatted_concepts = [(c['display_name'], c['works_api_url'], c['description'], c['image_url']) for c in top_n_concepts]
  return_concept = formatted_concepts[random.choice(range(len(formatted_concepts)))] # Randomly choose from the top_n_concepts
  return_concept_name, return_works_api_url, return_concept_description, image_thumbnail_url = return_concept[0], return_concept[1], return_concept[2], return_concept[3]
  print("CONCEPT: ", return_concept_name)
  print("WORKS API URL: ", return_works_api_url)
  print("IMAGE THUMBNAIL URL: ", image_thumbnail_url)
  # Retrieve and format scholarly works under this concept
  wr_json = requests.get(return_works_api_url + "&sort=cited_by_count:desc").json()
  top_n_works = wr_json["results"][0:n_works]
  works_dict = {}
  works_dict = {
      'title': [w['title'] for w in top_n_works],
      'url': [ f"<a href={ w['host_venue']['url'] } style='color: blue;' target='_blank' > {w['host_venue']['url']} </a>" for w in top_n_works ],
      'n_citations': [w['cited_by_count'] for w in top_n_works],
  }

  works_html = "<table> <tr><th style='border: 1px solid black;'> Title</th> <th style='border: 1px solid black;'> URL </th> <th style='border: 1px solid black;'> Citations </th> </tr>"
  for counter, w_link in enumerate(works_dict['url']):
    works_html +=  f"""<tr>
                        <td style='border: 1px solid black;'> { works_dict['title'][counter] } </td>
                        <td style='border: 1px solid black;'> {w_link} </td>
                          <td style='border: 1px solid black;'> { str(works_dict['n_citations'][counter]) } </td>
                       </tr>"""

  works_html += "</table>"

  # Format the image thumbnail if it exists

  img = None
  n_retries = 0
  if image_thumbnail_url is not None:
      while img == None and n_retries < 3: # Retry a couple of times

        # img_response = requests.get(image_thumbnail_url, stream=True)
        try:
          urllib.request.urlretrieve(image_thumbnail_url, "thumbnail-temp.png")
          img = Image.open("thumbnail-temp.png")
        except Exception:
          n_retries += 1
          img = None


  # Save information for PPO step on next iteration
  last_query_tensor, last_response_tensor = query_tensor, response_tensor


  return return_concept_name, img, return_concept_description, works_html, fig

try:
  demo = gr.Interface(
      fn=interact,
      inputs=[gr.Textbox(lines=3, label="S.T.E.M. topics you like"), gr.Slider(-1, 1, value=0, label="How much do you like this recommendation?")],
      outputs=[
        gr.Textbox(label="Recommended concept"),
        gr.Image(label="Concept thumbnail image"),
        gr.Textbox(label="Concept description"),
        gr.HTML(label="Academic works from this concept"),
        gr.Plot(label="Rewards over time"),
      ],
      # description="Enter a list of your interests. Rate the recommendations.",
  )
except Exception as e:
  print(e)

INTERESTS:  genomics, statistics, deep learning, chemistry,


In [ ]:
demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://d224cb76-f939-4b2b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


<ipython-input-37-5a02309c2ddc>:37: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


information technology, data science, cybersecurity, enterprise technology, computer vision, natural language processing, data engineering, my seminar, c me, open learn, genome paperontology learning, fast approach, new technology, biological parts
RECOMMENDATION =   my seminar
information technology, data science, cybersecurity, enterprise technology, computer vision, natural language processing, data engineeringscientific disciplines, genome informatics, socially mediated, and c, journal citation reports, web, distributed learning, excitability
RECOMMENDATION =   genome informatics
information technology, data science, cybersecurity, enterprise technology, computer vision, natural language processing, data engineering, mobile commotion, cog sensory, intelligent transportation systems, information access, s, computational algorithm, network algebra semantics,
RECOMMENDATION =   mobile commotion
information technology, data science, cybersecurity, enterprise technology, computer vision

<ipython-input-37-5a02309c2ddc>:37: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


information technology, data science, cybersecurity, enterprise technology, computer vision, natural language processing, data engineering, mechanistic, fluid, digital researcher, computer musicsoftware exploration, lab library, particularsscientific literatures, goodness off
RECOMMENDATION =   mechanistic
CONCEPT:  Mechanistic organic photochemistry
WORKS API URL:  https://api.openalex.org/works?filter=concepts.id:C50445383
IMAGE THUMBNAIL URL:  None
REWARD =  -0.5
Updating model params
   n_steps  rewards
0        1     0.25
1        2    -0.50
information technology, data science, cybersecurity, enterprise technology, computer vision, natural language processing, data engineering, philosophical approach, random field ising model, quality improvement paradigm, clarity, structure prediction, type system, c g
RECOMMENDATION =   philosophical approach
information technology, data science, cybersecurity, enterprise technology, computer vision, natural language processing, data engineerin

...

information, reinforcement learning, data science, cybersecurity, computer vision, natural language processing, data engineering,  cloud computing, mixed reality, super computing, word recognition, numeration system, mirror neuron, interprocedural analysis, transverse fluid
RECOMMENDATION =   mixed reality
CONCEPT:  Mixed reality
WORKS API URL:  https://api.openalex.org/works?filter=concepts.id:C206776904
IMAGE THUMBNAIL URL:  https://upload.wikimedia.org/wikipedia/commons/f/fb/Job_Simulator_mixed_reality_test_001.webm
REWARD =  0.5
Updating model params
    n_steps  rewards
0         1     0.25
1         2    -0.50
2         3     0.20
3         4     0.10
4         5    -0.06
5         6     0.01
6         7    -0.84
7         8    -0.22
8         9     0.16
9        10     0.40
10       11    -0.55
11       12    -0.47
12       13     0.20
13       14     0.50
14       15    -0.66
15       16    -0.32
16       17    -0.46
17       18     0.50
information, reinforcement learning, dat

<ipython-input-37-5a02309c2ddc>:37: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


information, reinforcement learning, data science, cybersecurity, computer vision, natural language processing, data engineering,  cloud computing, super computing, neuro consciousness, elementary chemistry, evolutionary trace method, a, lit review, n, super computing, extreme value
RECOMMENDATION =   super computing
information, reinforcement learning, data science, cybersecurity, computer vision, natural language processing, data engineering,  cloud computing, inferior temporal cortex, mechanism, opensourcesoftware exploration, work, inhomogeneity, digital resources, monographs, computations
RECOMMENDATION =   inferior temporal cortex
information, reinforcement learning, data science, cybersecurity, computer vision, natural language processing, data engineering,  cloud computing, hardware infrastructure, article, computer simulations, polymer project, minute constants, a design intelligence, elementary modes, wikipe da,
RECOMMENDATION =   hardware infrastructure
CONCEPT:  Message ori

<ipython-input-37-5a02309c2ddc>:37: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


information, reinforcement learning, data science, cybersecurity, computer vision, natural language processing, data engineering,  cloud computing, z mac, machine learning, paper a, working, compound, f we, intelligent systems, plasma physics, lattice theory
RECOMMENDATION =   z mac
information, reinforcement learning, data science, cybersecurity, computer vision, natural language processing, data engineering,  cloud computingparietal cortex, more, scientific disciplines, open learnings, science meta stuff, hyperbolic, a taxonomies,
RECOMMENDATION =   more
CONCEPT:  Randomized controlled trial
WORKS API URL:  https://api.openalex.org/works?filter=concepts.id:C168563851
IMAGE THUMBNAIL URL:  None
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d224cb76-f939-4b2b.gradio.live


In [ ]:
concept = interact("machine learning, offshore wind, ecology, sustainability, ocean engineering", last_step_reward=0.2)
print(concept[0], concept[2], concept[3], sep='\n')

machine learning, offshore wind, ecology, sustainability, ocean engineering, numerical physics, regulatory landscapemarkov model, neural integrators, product, molecular modelling
RECOMMENDATION =   numerical physics
CONCEPT:  Density matrix renormalization group
WORKS API URL:  https://api.openalex.org/works?filter=concepts.id:C29547527
IMAGE THUMBNAIL URL:  None
Density matrix renormalization group
numerical variational technique devised to obtain the low energy physics of quantum many-body systems with high accuracy
<table> <tr><th style='border: 1px solid black;'> Title</th> <th style='border: 1px solid black;'> URL </th> <th style='border: 1px solid black;'> Citations </th> </tr><tr>
                        <td style='border: 1px solid black;'> Density matrix formulation for quantum renormalization groups </td>
                        <td style='border: 1px solid black;'> <a href=None style='color: blue;' target='_blank' > None </a> </td>
                          <td style='border

### Results

#### Fine-tuned model

In [ ]:
i = "chemistry, deep learning, evolution, genomics, statistics, graph theory,"

In [ ]:
def inference(query_txt):
  # encode a query
  query_tensor = gpt2_tokenizer.encode(query_txt, return_tensors="pt")

  # Use GPT-2's next generated topic to retrieve n_concepts
  top_n_concepts = []

  # get model response
  response_tensor  = respond_to_batch(gpt2_model, query_tensor, txt_len= ( len(query_txt.split(" ")) + 10) )
  response_txt = gpt2_tokenizer.decode(response_tensor[0,:])

  return response_txt


for counter in range(5):
  print(f"{counter} {inference(i)}")

0  quant, ultra sensitivity, data source, domestic technologies, human processing, programming psychology, trends
1  software, integrating, frameworks, product overview, quantum algorithms, survey paper, molecular identification,
2  knowledge sharing, interesting, statistical analysis, a, omne tneuro, social inform
3  tool support, occur a, academic reference, feature discovery, multi process, design theory,
4  neuro psychdigitization, structural balancemultidimensional, disease propagation, research relationship, the


#### Non-fine-tuned model

In [ ]:
max_len = len(i.split(" ")) + 40
print(max_len)

for counter in range(5):
  print(f"{ counter } { generator1(i, num_return_sequences=1, max_length=max_len )[0]['generated_text'] }") # Set max length as the len of the input + (len of largest token in dataset)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


48


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0 chemistry, deep learning, evolution, genomics, statistics, graph theory, webpages, new technology, e, digital researcher, f, computer mediated, mobile elements, brain scanning, data mining software, human information interaction, user classification


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


1 chemistry, deep learning, evolution, genomics, statistics, graph theory, c humans, digital researcher, computer science educational, computational predicition, data source, c, medial frontal cortex, fagins algorithm, software architecture, intelligent


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


2 chemistry, deep learning, evolution, genomics, statistics, graph theory, research relationship, a, knowledge representation, science politics, molecular methods, t, mobile components, research relationship, v, network neural, software exploration, ontology


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


3 chemistry, deep learning, evolution, genomics, statistics, graph theory, cognitive control interface, research relationship, network based, data source, numerical physics, neural integrators, web trends, in, neural net, knowledge sharingmobile networks
4 chemistry, deep learning, evolution, genomics, statistics, graph theory, computational sociology, research paper, biological modelling, science communication, cgraph theory, functional analyses, n, molecular mechanics, information soc, s, e, medial


#### Scratch

In [ ]:
n_step

1

In [ ]:
# import numpy.array as array

# {'objective/kl': array(1.289462, dtype=float32), 'objective/kl_dist': array([[-0.18468547,  0.02913914,  0.74619746,  0.20198345,  0.28988886,
#         -0.18992865, -0.288383  ,  0.25420332,  0.02143562,  0.18235779,
#          0.4640088 , -0.23675537]], dtype=float32), 'objective/logprobs': array([[-9.7589660e-01, -1.0072879e-03, -1.0765718e+00, -8.2480116e+00,
#         -3.3704519e+00, -1.0414710e+00, -6.3635521e+00, -5.6039419e+00,
#         -1.7408788e+00, -9.2117701e+00, -6.0865836e+00, -7.2170386e+00]],
#       dtype=float32), 'objective/ref_logprobs': array([[-0.7912111 , -0.03014643, -1.8227693 , -8.449995  , -3.6603408 ,
#         -0.85154235, -6.075169  , -5.858145  , -1.7623144 , -9.394128  ,
#         -6.5505924 , -6.9802833 ]], dtype=float32), 'objective/kl_coef': 0.19996000359980798, 'objective/entropy': array(50.937176, dtype=float32), 'ppo/mean_non_score_reward': array(-0.2578408, dtype=float32), 'ppo/loss/policy': array([3.0795732e-07], dtype=float32), 'ppo/loss/value': array([0.55313087], dtype=float32), 'ppo/loss/total': array([0.0553134], dtype=float32), 'ppo/policy/entropy': array([4.922212], dtype=float32), 'ppo/policy/approxkl': array([0.], dtype=float32), 'ppo/policy/policykl': array([0.], dtype=float32), 'ppo/policy/clipfrac': array([0.]), 'ppo/policy/advantages': array([ 0.55474675,  2.389362  ,  0.22772089,  0.03694599, -0.5207685 ,
#         0.7164826 , -0.1692832 , -0.7495509 , -0.7058894 , -1.0186545 ,
#        -1.310441  ,  0.5493255 ], dtype=float32), 'ppo/policy/advantages_mean': array([-3.0795732e-07], dtype=float32), 'ppo/policy/ratio': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32), 'ppo/returns/mean': array([-0.8171821], dtype=float32), 'ppo/returns/var': array([0.01461094], dtype=float32), 'ppo/val/vpred': array([0.19017708], dtype=float32), 'ppo/val/error': array([1.1062617], dtype=float32), 'ppo/val/clipfrac': array([0.]), 'ppo/val/mean': array([0.19017708], dtype=float32), 'ppo/val/var': array([0.07251107], dtype=float32), 'ppo/val/var_explained': array([-74.714615], dtype=float32), 'time/ppo/forward_pass': 0.29840755462646484, 'time/ppo/compute_rewards': 0.00018453598022460938, 'time/ppo/optimize_step': 1.1635761260986328, 'time/ppo/calc_stats': 0.0008087158203125, 'time/ppo/total': 1.4636282920837402}\
#        .keys()

In [ ]:
# machine learning, deep learning, graph theory, womens health